# Build vocabulary and tokenizer
본 노트에서는 자연어처리 관련 딥러닝 모형학습을 위한 필수적인 전처리중 하나로 training corpus에 존재하는 token들의 집합인 **vocabulary**를 만들어 봅니다. vocabulary 구성 자체는 미리 제공하는 `Vocab` class를 활용해봅니다. 또한 미리 제공하는 `Tokenizer` class를 같이 활용하여, 효율적인 전처리를 어떻게 할 수 있는 지 확인합니다.

### Setup

In [1]:
import sys
import itertools
import pandas as pd
from pathlib import Path
from pprint import pprint
from typing import List
from collections import Counter
from utils import Vocab, Tokenizer

### Load dataset

In [2]:
data_dir = Path.cwd() / 'data'
list_of_dataset = list(data_dir.iterdir())
pprint(list_of_dataset)

[PosixPath('/root/Documents/archive/strnlp/exercise/data/train.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/validation.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/test.txt')]


In [3]:
tr_dataset = pd.read_csv(list_of_dataset[1], sep='\t')
tr_dataset.head()

,document,label
0,60년대 각본을 가져다가 만든거 같이 진부하고 춤도 이박사 춤이 더 나을거 같음,0
1,아 신난다~,0
2,이게 한쿡영화의 현실..,0
3,박철수가 만들었다니 ...,0
4,와 항상 금요일이 기대되네요ㅠ 몬스타 화이팅 !,1


### Split training corpus and count each tokens
앞선 `eda.ipynb` 노트에서 활용헌 `split_eojeol` function을 이용, **training corpus를 sequence of tokens의 형태로 변환하고, training corpus에서 token의 출현빈도를 계산합니다.**

In [4]:
# 문장을 어절기준으로 보는 split_fn을 작성
def split_eojeol(s: str) -> List[str]:
    return s.split(' ')

In [5]:
training_corpus = tr_dataset['document'].apply(lambda sen: split_eojeol(sen)).tolist()
pprint(training_corpus[:5])

[['60년대',
  '각본을',
  '가져다가',
  '만든거',
  '같이',
  '진부하고',
  '춤도',
  '이박사',
  '춤이',
  '더',
  '나을거',
  '같음'],
 ['아', '신난다~'],
 ['이게', '한쿡영화의', '현실..'],
 ['박철수가', '만들었다니', '...'],
 ['와', '항상', '금요일이', '기대되네요ㅠ', '몬스타', '화이팅', '!']]


In [6]:
count_tokens = Counter(itertools.chain.from_iterable(training_corpus))
print(len(count_tokens))

98220


### Build vocab
`min_freq`를 설정하고, training corpus에서 출현빈도가 `min_freq`보다 낮은 token을 제외하고, `Vocab` class를 이용하여 Vocabulary를 구축합니다. `min_freq`보다 낮은 token들은 `unknown`으로 처리됩니다.

1. `list_of_tokens`을 만듭니다. `list_of_tokens`은 `min_freq`보다 낮은 token들을 모아놓은 `list`입니다.
2. `vocab`을 만듭니다. `list_of_tokens`를 parameter로 전달받습니다.

In [7]:
min_freq = 5
list_of_tokens = [token for token in count_tokens.keys() if count_tokens.get(token) > min_freq]
print(len(list_of_tokens))

4381


`Vocab` class는 `list_of_tokens`를 parameter로 전달받아 아래와 같이 생성합니다.

In [8]:
vocab = Vocab(list_of_tokens=list_of_tokens, bos_token=None, eos_token=None, unknown_token_idx=0)

### How to use `Vocab`
`list_of_tokens`로 생성한 `Vocab` class의 instance인 `vocab`은 아래와 같은 멤버들을 가지고 있습니다.

In [9]:
help(Vocab)

Help on class Vocab in module utils:

class Vocab(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, list_of_tokens=None, padding_token='<pad>', unknown_token='<unk>', bos_token='<bos>', eos_token='<eos>', reserved_tokens=None, unknown_token_idx=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  to_indices(self, tokens:Union[str, List[str]]) -> Union[int, List[int]]
 |  
 |  to_tokens(self, indices:Union[int, List[int]]) -> Union[str, List[str]]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bos_token
 |  
 |  embedding
 |  
 |  eos_token
 |  
 |  idx_to_token
 |  
 |  padding_token
 |  
 |  token_to_idx
 |  
 |  unknown_token



In [10]:
# padding_token, unknown_token, eos_token, bos_token
print(vocab.padding_token)
print(vocab.unknown_token)
print(vocab.eos_token)
print(vocab.bos_token)

<pad>
<unk>
None
None


In [11]:
# token_to_idx
print(vocab.token_to_idx)

{'<unk>': 0, '<pad>': 1, '각본을': 2, '같이': 3, '진부하고': 4, '더': 5, '같음': 6, '아': 7, '이게': 8, '...': 9, '와': 10, '항상': 11, '화이팅': 12, '!': 13, '이제': 14, '나이': 15, '예전': 16, '같지': 17, '않아': 18, '아~': 19, '그리고': 20, '마지막': 21, '장면': 22, '그닥': 23, '웃기지도': 24, '포스터': 25, '좀': 26, '듯..': 27, '일단': 28, '다': 29, '여주': 30, '미친듯이': 31, '정말': 32, '연기력에': 33, '박수를': 34, '완벽하게': 35, '잘': 36, '명작이다': 37, '전편의': 38, '기대하고': 39, '뭐지?': 40, '훈훈한': 41, '그저': 42, '뿐': 43, '이': 44, '영화는': 45, '너무': 46, '연기': 47, '할': 48, '필요': 49, '없는': 50, '영화를': 51, '선택한': 52, '로맨틱': 53, '제발': 54, '이건': 55, '공포가': 56, '공포를': 57, '내용도': 58, '없고': 59, '기분만': 60, '영화': 61, '작품': 62, '캐스팅': 63, '이유없이': 64, '싶은': 65, '아이들의': 66, '장난': 67, '슬픈': 68, '이걸': 69, '끝까지': 70, '본': 71, '내가': 72, '대단하다': 73, '1점도': 74, '아까움': 75, '20년이': 76, '그냥': 77, '시간': 78, '많은': 79, '것을': 80, '생각하게': 81, '하는': 82, '1편을': 83, '반도': 84, '못': 85, '차라리': 86, '볼만함': 87, '귀신': 88, '평점이': 89, '이영화가': 90, '왜': 91, '이해가': 92, '유치한': 93, '후반부에': 94, '아는': 95,

In [12]:
# idx_to_token
print(vocab.idx_to_token)

{0: '<unk>', 1: '<pad>', 2: '각본을', 3: '같이', 4: '진부하고', 5: '더', 6: '같음', 7: '아', 8: '이게', 9: '...', 10: '와', 11: '항상', 12: '화이팅', 13: '!', 14: '이제', 15: '나이', 16: '예전', 17: '같지', 18: '않아', 19: '아~', 20: '그리고', 21: '마지막', 22: '장면', 23: '그닥', 24: '웃기지도', 25: '포스터', 26: '좀', 27: '듯..', 28: '일단', 29: '다', 30: '여주', 31: '미친듯이', 32: '정말', 33: '연기력에', 34: '박수를', 35: '완벽하게', 36: '잘', 37: '명작이다', 38: '전편의', 39: '기대하고', 40: '뭐지?', 41: '훈훈한', 42: '그저', 43: '뿐', 44: '이', 45: '영화는', 46: '너무', 47: '연기', 48: '할', 49: '필요', 50: '없는', 51: '영화를', 52: '선택한', 53: '로맨틱', 54: '제발', 55: '이건', 56: '공포가', 57: '공포를', 58: '내용도', 59: '없고', 60: '기분만', 61: '영화', 62: '작품', 63: '캐스팅', 64: '이유없이', 65: '싶은', 66: '아이들의', 67: '장난', 68: '슬픈', 69: '이걸', 70: '끝까지', 71: '본', 72: '내가', 73: '대단하다', 74: '1점도', 75: '아까움', 76: '20년이', 77: '그냥', 78: '시간', 79: '많은', 80: '것을', 81: '생각하게', 82: '하는', 83: '1편을', 84: '반도', 85: '못', 86: '차라리', 87: '볼만함', 88: '귀신', 89: '평점이', 90: '이영화가', 91: '왜', 92: '이해가', 93: '유치한', 94: '후반부에', 95: '아는',

In [13]:
# to_indices
example_sentence = tr_dataset['document'][0]
tokenized_sentence = split_eojeol(example_sentence)
transformed_sentence = vocab.to_indices(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

60년대 각본을 가져다가 만든거 같이 진부하고 춤도 이박사 춤이 더 나을거 같음
['60년대', '각본을', '가져다가', '만든거', '같이', '진부하고', '춤도', '이박사', '춤이', '더', '나을거', '같음']
[0, 2, 0, 0, 3, 4, 0, 0, 0, 5, 0, 6]


In [14]:
# to_tokens
print(vocab.to_tokens(transformed_sentence))

['<unk>', '각본을', '<unk>', '<unk>', '같이', '진부하고', '<unk>', '<unk>', '<unk>', '더', '<unk>', '같음']


### How to use `Tokenizer`
위의 `Vocab` class의 활용 형태를 보면 `split_fn`으로 활용하는 `split_eojeol` function의 결과를 input을 기본적으로 받습니다. `Vocab` class의 instance와 `split_fn`으로 활용하는 `split_eojeol` function을 input으로 받아 전처리를 해주는 `Tokenizer` class를 활용할 수 있습니다.

In [15]:
help(Tokenizer)

Help on class Tokenizer in module utils:

class Tokenizer(builtins.object)
 |  Tokenizer class
 |  
 |  Methods defined here:
 |  
 |  __init__(self, vocab:utils.Vocab, split_fn:Callable[[str], List[str]], pad_fn:Callable[[List[int]], List[int]]=None) -> None
 |      Instantiating Tokenizer class
 |      
 |      Args:
 |          vocab (model.utils.Vocab): the instance of model.utils.Vocab created from specific split_fn
 |          split_fn (Callable): a function that can act as a splitter
 |          pad_fn (Callable): a function that can act as a padder
 |  
 |  split(self, string:str) -> List[str]
 |  
 |  split_and_transform(self, string:str) -> List[int]
 |  
 |  transform(self, list_of_tokens:List[str]) -> List[int]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

In [16]:
tokenizer = Tokenizer(vocab, split_eojeol)

In [17]:
# split, transform, split_and_transform
example_sentence = tr_dataset['document'][0]
tokenized_sentence = tokenizer.split(example_sentence)
transformed_sentence = tokenizer.transform(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)
print(tokenizer.split_and_transform(example_sentence))

60년대 각본을 가져다가 만든거 같이 진부하고 춤도 이박사 춤이 더 나을거 같음
['60년대', '각본을', '가져다가', '만든거', '같이', '진부하고', '춤도', '이박사', '춤이', '더', '나을거', '같음']
[0, 2, 0, 0, 3, 4, 0, 0, 0, 5, 0, 6]
[0, 2, 0, 0, 3, 4, 0, 0, 0, 5, 0, 6]
